# Converting ASDF Rotations to Quaternions

This notebook shows the same thing as the
[notebook about rotation matrices](rotation-matrices.ipynb),
just using quaternions instead of rotation matrices.
For more detailed explanations, have a look over there.

You might be tempted to use the equations from
[Wikipedia](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles#Euler_angles_(in_3-2-1_sequence)_to_quaternion_conversion),
but those use different conventions for axes and angles!
The resulting equations will have a similar structure but will not be quite identical.

With the code below, any convention can be calculated by adapting

* the pairing of angles with their corresponding axes
* the sign of angles (or direction of axes) according to handedness
* the order of combining the individual axis/angle quaternions

In [ ]:
import sympy as sp

In [ ]:
from sympy.algebras import Quaternion

In [ ]:
alpha, beta, gamma = sp.symbols('alpha beta gamma')

## Azimuth: Rotation around the z-Axis

In [ ]:
q_z = Quaternion.from_axis_angle((0, 0, 1), alpha)
q_z

Example:
Rotating the y unit vector (i.e. “looking north”)
by 90 degrees to the left:

In [ ]:
Quaternion.rotate_point((0, 1, 0), q_z.subs(alpha, sp.pi / 2))

As expected, this yields the negative x unit vector, which points westwards.

## Elevation: Rotation around the (local) x-Axis

In [ ]:
q_x = Quaternion.from_axis_angle((1, 0, 0), beta)
q_x

Example:
Applying 90 degrees of elevation to
the y unit vector:

In [ ]:
Quaternion.rotate_point((0, 1, 0), q_x.subs(beta, sp.pi / 2))

As expected, this yields a vector pointing up.

## Roll: Rotation around the (local) y-Axis

In [ ]:
q_y = Quaternion.from_axis_angle((0, 1, 0), gamma)
q_y

Example: Applying a roll angle of 90 degrees to a vector pointing up:

In [ ]:
Quaternion.rotate_point((0, 0, 1), q_y.subs(gamma, sp.pi / 2))

As expected, this yields a vector pointing east.

## Combining all Axes

This is easy,
we only have to make sure to use the right order.
As with rotation matrices,
you should read this from right to left
(first *roll*, then *elevation*, then *azimuth*):

In [ ]:
q = q_z * q_x * q_y
q

If you want to copy-paste this:

In [ ]:
print(q)

But you should probably pre-calculate the used terms
in order to avoid repeated evaluation of the same functions.
You could try something like this, for example:

In [ ]:
q.subs([
    (sp.sin(alpha/2), sp.symbols('s_alpha')),
    (sp.sin(beta/2), sp.symbols('s_beta')),
    (sp.sin(gamma/2), sp.symbols('s_gamma')),
    (sp.cos(alpha/2), sp.symbols('c_alpha')),
    (sp.cos(beta/2), sp.symbols('c_beta')),
    (sp.cos(gamma/2), sp.symbols('c_gamma')),
])

In [ ]:
print(_)

## Quaternion to Rotation Matrix

Just to make sure the result is the same as in the
[notebook about rotation matrices](rotation-matrices.ipynb#Combining-all-Axes),
let's calculate the rotation matrix from our quaternion.

For some reason, SymPy seems to need two simplification steps for this ...

In [ ]:
R = sp.trigsimp(sp.trigsimp(q.to_rotation_matrix()))
R

## Quaternion to ASDF rotations

Again, please note that the equations from
[Wikipedia](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles#Quaternion_to_Euler_angles_(in_3-2-1_sequence)_conversion)
use different conventions for axes and angles.

We already know how to convert a rotation matrix to ASDF angles,
and we know how to convert a quaternion to a rotation matrix,
so let's try that:

In [ ]:
a, b, c, d = sp.symbols('a:d')

In [ ]:
sp.simplify(sp.Quaternion(a, b, c, d).to_rotation_matrix())

Since we assume a unit quaternion, all the denominators are actually 1.

In [ ]:
Rq = sp.simplify(sp.Quaternion(a, b, c, d).to_rotation_matrix().subs(a**2 + b**2 + c**2 + d**2, 1))
Rq

The [notebook about rotation matrices](rotation-matrices.ipynb#Rotation-Matrix-to-Angles)
shows how to obtain $\alpha$, $\beta$ and $\gamma$ from this matrix.

We can get $\alpha$ from the top middle and the central element:

In [ ]:
sp.atan2(-Rq[0, 1], Rq[1, 1])

In [ ]:
print(_)

The bottom middle element provides $\beta$:

In [ ]:
sp.asin(Rq[2, 1])

In [ ]:
print(_)

<div class="alert alert-info">

**Note:**

As mentioned in the
[notebook about rotation matrices](rotation-matrices.ipynb#Rotation-Matrix-to-Angles),
the argument of the `asin()` function has to be in the domain `[-1.0; 1.0]`.

Make sure to handle this case,
e.g. by re-normalizing the quaternion.

</div>

Finally, $\gamma$ can be obtained from the bottom left and right elements:

In [ ]:
sp.atan2(-Rq[2, 0], Rq[2, 2])

In [ ]:
print(_)

### Gimbal Lock

As shown in the
[notebook about rotation matrices](rotation-matrices.ipynb#Gimbal-Lock),
there is a problem when $\beta = \pm 90$ degrees.

For $\beta = 90$ degrees (which means $2ab+2cd = 1$),
we can obtain a value for $\alpha + \gamma$:

In [ ]:
sp.atan2(Rq[0, 2], -Rq[1, 2])

In [ ]:
print(_)

If we for example choose this value to be $\alpha$,
this will result in $\gamma = 0$.

Alternatively, we can use this expression:

In [ ]:
sp.atan2(Rq[1, 0], Rq[0, 0])

In [ ]:
print(_)

For $\beta = -90$ degrees (which means $2ab+2cd = -1$),
we can use the following expression for $\alpha + \gamma$:

In [ ]:
sp.atan2(-Rq[0, 2], Rq[1, 2])

In [ ]:
print(_)

Again, if we for example choose this value to be $\alpha$,
this will result in $\gamma = 0$.

Alternatively, we can use this expression:

In [ ]:
sp.atan2(Rq[1, 0], Rq[0, 0])

In [ ]:
print(_)